In [1]:
import pandas as pd

In [2]:
#Specify dtype option on import or set low_memory=False.란 에러가 발생하여 dtype = 'unicode'를추가
공시 = pd.read_csv('사용디비.csv',encoding = 'euc-kr', dtype='unicode')
공시.head()

#pandas.query로는 데이터프레임 내의 텍스트 데이터 검색가능 
data = 공시.query('구분.str.contains("코스닥시장") or 구분.str.contains("유가증권")',engine = 'python')

#2010년도 부터의 데이터만 따로 추출
filter_data = data.loc[(data['날짜'] > '2010-01-01')] 

filter_data = filter_data.drop(['Unnamed: 0'],axis = 1)

In [3]:
#날짜별로 정렬
#groupby 이용시 적용해줄 함수 설정 
def top(df):
    return df.sort_values(by='날짜')

#카테고리별로 묶어준 데이터 날짜 순으로 정리.
cat_data = filter_data.groupby('CategorySeq').apply(top)

#groupby해주는 과정에 같이 병합된 인덱스를 제거
#multi index에서 index 하나를 drop 할시에 이용
cat_data = cat_data.reset_index(level=1, drop=True)
cat_data.head(1)

,보고서키,날짜,구분,CategorySeq,보고서명,기재정정여부,자율공시여부,종목키,종목명,종목코드,...,주권비율.1,주권직전수.1,주권직전비율.1,주권증가수.1,주권증가비율.1,의결권수,의결권직전수,의결권증감수,변동사유,변경사유
CategorySeq,,,,,,,,,,,,,,,,,,,,,
A001,20100128000200,2010-01-28 09:13:00,코스닥시장,A001,[첨부추가]사업보고서(2009.10),0,0,00176835,농우바이오,054050,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 보고서 카테고리 확인용

In [4]:
카테고리 = pd.read_csv('보고서카테고리.csv',encoding = 'euc-kr')
cat_map = 카테고리[['CategorySeq','ReportName']]
cat_map.head()

,CategorySeq,ReportName
0,A,정기공시
1,A001,사업보고서
2,A001001,사업보고서
3,A001002,사업보고서(외국기업이외의외국법인등의경우)
4,A001003,사업보고서ㆍ반기보고서ㆍ분기보고서(외국기업의경우)


### 주가등락률 표준편차(pandas를 이용해서)

In [5]:
공시주가 = cat_data[['일봉주가등락률','일주후주가등락률']]
#결측값제거
공시주가 = 공시주가.dropna()
공시주가 = 공시주가.astype({'일봉주가등락률' : 'float','일주후주가등락률':'float'})

공시주가_표준편차 = 공시주가.groupby(['CategorySeq']).std()
공시주가_표준편차.rename(columns ={'일봉주가등락률':'일봉주가등락률_표준편차', '일주후주가등락률':'일주후주가등락률_표준편차'},inplace = True)
공시주가_표준편차.head()

,일봉주가등락률_표준편차,일주후주가등락률_표준편차
CategorySeq,,
A001,0.035776,0.088971
A001005,0.110581,0.312778
A002,0.039263,0.089441
A002003,0.010116,0.016186
A003,0.037527,0.082283


### 주가등락률 평균

In [6]:
공시주가_평균 = 공시주가.groupby(['CategorySeq']).mean()
공시주가_평균.rename(columns ={'일봉주가등락률':'일봉주가등락률_평균', '일주후주가등락률':'일주후주가등락률_평균'},inplace = True)
공시주가_평균.head()

,일봉주가등락률_평균,일주후주가등락률_평균
CategorySeq,,
A001,0.003201,0.016866
A001005,0.004676,0.105779
A002,-0.003437,-0.008644
A002003,-0.007153,0.011445
A003,-0.002570,-0.003936


### 무조건 상승거나 하락하는 공시 구해주기

In [7]:
비교용 = 공시주가.copy()
비교용 = 비교용.reset_index(level=0, drop=False)
비교용

,CategorySeq,일봉주가등락률,일주후주가등락률
0,A001,-0.025253,-0.020202
1,A001,0.007264,0.016949
2,A001,-0.016667,-0.087778
3,A001,-0.025997,-0.053726
4,A001,0.001266,-0.017722
...,...,...,...
966187,J005033,-0.217778,-0.525333
966188,J005033,0.300000,0.000000
966189,J005033,0.300000,0.000000
966190,J005033,0.295918,1.178571


In [8]:
일주_최소 = 비교용[비교용['CategorySeq'] == 'A001'].min().일주후주가등락률_평균
일주_최대 = 비교용[비교용['CategorySeq'] == 'A001'].max().일주후주가등락률_평균

AttributeError: 'Series' object has no attribute '일주후주가등락률_평균'

In [ ]:
for each in list(비교용['CategorySeq'].unique()):
    if (비교용[비교용['CategorySeq'] == each].min().일주후주가등락률 >0 ):
        print(each,비교용[비교용['CategorySeq'] == each].min().일주후주가등락률_평균)

# 표준편차와 평균등락률 데이터 병합

In [ ]:
공시_편차 = pd.merge(공시주가_표준편차,공시주가_평균,left_index = True ,right_index = True)

#데이터프레임 병합 
data = pd.merge(공시_편차, cat_map, left_index=True, right_on='CategorySeq')
#data = data.set_index(['CategorySeq'])
data

In [ ]:
#상승확률과 하락확률
일봉_확률 = data.copy() 
상승확률 = []
하락확률 = []
카테고리 = []   

for each in list(data['CategorySeq'].unique()):
    해당공시 = data[(data['CategorySeq'] == each)]
    카테고리.append(each)
    상승확률.append(len(해당공시[해당공시['일봉주가등락률_평균'] > 0]) / len(해당공시))
    하락확률.append(len(해당공시[해당공시['일봉주가등락률_평균'] < 0]) / len(해당공시))    

확률 = pd.DataFrame(list(zip(카테고리,상승확률,하락확률)),columns = ['CategorySeq','해당공시_상승확률','해당공시_하락확률'])
확률